# 2주차 과제
updated 0912   
파이썬과 mysql 연동 후 데이터 가져오기  

## Python 과제
1. 엑셀 파일 db_score.xlsx 를 입력파일로 받아서 dataframe으로 읽어 들이고,   
이 데이터들을 mysql db_score 테이블을 생성하고 모두 튜플로 추가하는 python 프로그램을 작성할 것.  
2. db_score 테이블로 부터 midterm(중간고사) 와 final(기말고사) 가 모두 20점 이상인 학생의  
학번, 중간고사, 기말고사를 학번 순으로 출력하는 python 프로그램을 작성할 것

> - 1,2는 각각 python 프로그램으로 작성할 것  
> - 제출재용 : 개발설명, 소스코드, 실행 결과 캡쳐  
> -  제출마감 : 9월 14일(월) 자정

In [7]:
# ready 
import pandas as pd
import numpy as np
import pymysql

import warnings
warnings.filterwarnings(action='ignore')

from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_info_columns', 500)

## Q1. 
엑셀 파일 db_score.xlsx 를 입력파일로 받아서 dataframe으로 읽어 들이고,
이 데이터들을 mysql db_score 테이블을 생성하고 모두 튜플로 추가하는 python 프로그램을 작성할 것.

### step1. Load data

In [8]:
data = pd.read_excel('data/db_score.xlsx')
data.head()

,sno,attendance,homework,discussion,midterm,final,score,grade
0,1,7.73,19.37,0,35.00,30.10,92.20,A
1,2,7.73,20.00,0,34.12,30.10,91.95,A
2,3,7.73,19.37,0,34.30,29.75,91.15,A
3,4,8.00,18.12,2,31.50,29.05,88.67,A
4,5,8.00,20.00,2,33.25,21.35,84.60,A


### step2. python 에서 mysql 테이블 만들기

In [19]:
# db에 db_score 테이블 생성
# university.db_score
def create_table():
    conn = pymysql.connect(host='localhost', user='root', 
                           password='1234',db = 'university', charset='utf8')
    #curs = conn.cursor(pymysql.cursors.DictCursor)

    sql = '''
    CREATE TABLE db_score
    (
    sno INT NOT NULL,
    attendance FLOAT NOT NULL,
    homework FLOAT NOT NULL,
    discussion INT NOT NULL,
    midterm FLOAT NOT NULL,
    final FLOAT NOT NULL,
    score FLOAT NOT NULL,
    grade VARCHAR(2) NOT NULL
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8
    '''
    
    try:
        with conn.cursor() as cursor:
            cursor.execute(sql)
        conn.commit()
    finally:
        conn.close()

In [20]:
create_table()

In [24]:
# data db_score 에 파일 넣기
vals=tuple([tuple(x) for x in data.values])

try:
    conn = pymysql.connect(host='localhost', user='root', 
                           password='1234',db = 'university', charset='utf8')
    
    curs=conn.cursor(pymysql.cursors.DictCursor)
    
    sql=f"INSERT INTO university.db_score (sno, attendance, homework, discussion, midterm, final, score, grade) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"
    
    curs.executemany(sql, vals)
    conn.commit()
    print(curs.rowcount, "records were inserted")
    
finally:
    conn.close()

92 records were inserted


In [25]:
# 확인
conn = pymysql.connect(host='localhost', user='root', 
                           password='1234',db = 'university', charset='utf8')
try:
    with conn.cursor() as curs:  
        sql = "select * from db_score"
        curs.execute(sql)
        result = curs.fetchall()
        
        for i in result:
            print(i)
finally:
    conn.close()

(1, 7.73, 19.37, 0, 35.0, 30.1, 92.2, 'A')
(2, 7.73, 20.0, 0, 34.12, 30.1, 91.95, 'A')
(3, 7.73, 19.37, 0, 34.3, 29.75, 91.15, 'A')
(4, 8.0, 18.12, 2, 31.5, 29.05, 88.67, 'A')
(5, 8.0, 20.0, 2, 33.25, 21.35, 84.6, 'A')
(6, 8.0, 19.37, 0, 24.5, 32.2, 84.07, 'A')
(7, 8.0, 19.37, 2, 23.8, 28.7, 81.87, 'A')
(8, 8.0, 16.87, 0, 28.87, 25.9, 79.64, 'A')
(9, 8.0, 19.37, 0, 29.75, 21.7, 78.82, 'A')
(10, 7.73, 20.0, 0, 31.5, 19.25, 78.48, 'A')
(11, 8.0, 19.37, 0, 29.57, 21.35, 78.29, 'A')
(12, 6.66, 19.37, 0, 31.5, 20.65, 78.18, 'A')
(13, 8.0, 19.37, 0, 29.75, 21.0, 78.12, 'A')
(14, 8.0, 20.0, 0, 31.5, 18.55, 78.05, 'A')
(15, 7.73, 18.12, 0, 28.0, 23.1, 76.95, 'A')
(16, 8.0, 19.37, 0, 24.5, 24.85, 76.72, 'A')
(17, 7.46, 20.0, 0, 30.62, 18.55, 76.63, 'A')
(18, 8.0, 19.37, 0, 30.62, 18.55, 76.54, 'A')
(19, 8.0, 19.37, 0, 24.5, 24.5, 76.37, 'A')
(20, 8.0, 19.37, 0, 28.0, 20.65, 76.02, 'A')
(21, 8.0, 19.37, 0, 35.0, 12.95, 75.32, 'B')
(22, 8.0, 16.87, 0, 35.0, 15.05, 74.92, 'B')
(23, 8.0, 19.37, 0, 

## Q2.
db_score 테이블로 부터 midterm(중간고사) 와 final(기말고사) 가 모두 20점 이상인 학생의
학번, 중간고사, 기말고사를 학번 순으로 출력하는 python 프로그램을 작성할 것


In [28]:
# 연결
conn = pymysql.connect(host='localhost', user='root', 
                           password='1234',db = 'university', charset='utf8')

# 해당 학생 데이터 불러들이기
try:
    with conn.cursor() as curs:  
        sql = "SELECT sno, midterm, final FROM db_score \
                WHERE midterm>=20 and final>=20 \
                ORDER BY sno"
        curs.execute(sql)
        result = curs.fetchall()
        # 확인
        for i in result:
            print(i)
finally:
    conn.close()     # 연결 닫기

(1, 35.0, 30.1)
(2, 34.12, 30.1)
(3, 34.3, 29.75)
(4, 31.5, 29.05)
(5, 33.25, 21.35)
(6, 24.5, 32.2)
(7, 23.8, 28.7)
(8, 28.87, 25.9)
(9, 29.75, 21.7)
(11, 29.57, 21.35)
(12, 31.5, 20.65)
(13, 29.75, 21.0)
(15, 28.0, 23.1)
(16, 24.5, 24.85)
(19, 24.5, 24.5)
(20, 28.0, 20.65)
(27, 22.75, 22.05)
(29, 26.25, 22.4)
(37, 24.5, 20.3)


In [35]:
df = pd.DataFrame(result, columns=['sno', 'midterm', 'final'])
df

,sno,midterm,final
0,1,35.00,30.10
1,2,34.12,30.10
2,3,34.30,29.75
3,4,31.50,29.05
4,5,33.25,21.35
5,6,24.50,32.20
6,7,23.80,28.70
7,8,28.87,25.90
8,9,29.75,21.70
9,11,29.57,21.35
